# Lab 04 · Structured JSON and Validation

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Git refresher
Before changing backend code, verify your repository state so work-in-progress branches stay clean.

```bash
git init .
git add .
git commit -m "my first commit to ..."
git status
git checkout -b lab04-structured-json
git add ai-web/backend/app
git commit -m "Add planner service"
git push origin lab04-structured-json
```

> The branch name above is a suggestion—feel free to align it with your team's conventions.

## Objectives
- Introduce a dedicated `app/schemas/planner.py` module for nested planner models.
- Enforce validation and automatic repair for planner payloads received from any client.
- Expose `/planner/plan` and `/planner/plan/validate` endpoints that integrate with the existing FastAPI application.

## Why planners matter for the AI Web app

Our FastAPI backend already powers the AI web experience you built in Labs 02–03. The "planner" components in this lab give the agents a predictable contract so the UI and future services can rely on structured steps, owners, and rationales. By validating planner payloads at the schema and service layers, we guarantee that the release readiness agent can surface actionable plans without manual clean-up.

- **Schemas** keep every planner output aligned with the nested JSON that the React app expects.
- **Services** turn loosely structured model responses into clean task lists tied to product, engineering, and QA stakeholders.
- **Routers** expose `/planner` endpoints so the existing UI and other agents can request plans, review validation warnings, and store them alongside release readiness recommendations.

Keep this workflow in mind as you follow the lab steps—the code you author here plugs directly into the shared backend so the app can scale beyond simple demos.

## What will be learned
- Authoring reusable Pydantic schemas that scale with additional planner features.
- Designing service-layer helpers that sanitize loosely structured JSON.
- Surfacing validation utilities through FastAPI routers so the React app can consume them.

## Prerequisites & install
The following commands are intended for local execution. Ensure the virtual environment from Lab 01 remains active.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install pydantic
```

## Step-by-step tasks
You will create schema, service, and router layers that align with the architecture established in previous labs.

### Step 1: Planner schema module
Create `app/schemas/planner.py` so planners share consistent nested structures across services and routers.

In [ ]:
from pathlib import Path

schema_dir = Path('ai-web/backend/app/schemas')
schema_dir.mkdir(parents=True, exist_ok=True)
(schema_dir / '__init__.py').write_text('"""Pydantic schema definitions shared across the FastAPI app."""\n')
(schema_dir / 'planner.py').write_text('"""Structured planner schemas shared by labs and services."""\n\nfrom __future__ import annotations\n\nfrom datetime import datetime\nfrom typing import Literal\n\nfrom pydantic import BaseModel, Field, field_validator, model_validator\n\n\nclass PlanAudience(BaseModel):\n    """Describe who the plan is designed for so UI copy can adjust."""\n\n    role: str = Field(..., min_length=2, max_length=64)\n    experience_level: Literal["beginner", "intermediate", "advanced"]\n\n\nclass PlanStep(BaseModel):\n    """Individual step in the generated plan."""\n\n    title: str = Field(..., min_length=3, max_length=120)\n    description: str = Field(..., min_length=10, max_length=500)\n    owner: str = Field(..., min_length=2, max_length=60)\n    duration_minutes: int = Field(..., ge=5, le=240)\n    acceptance_criteria: list[str] = Field(default_factory=list)\n\n    @field_validator("acceptance_criteria")\n    @classmethod\n    def _trim_criteria(cls, value: list[str]) -> list[str]:\n        """Remove empty acceptance criteria entries submitted by the client."""\n\n        return [item.strip() for item in value if item.strip()]\n\n\nclass Plan(BaseModel):\n    """Top-level plan returned to the frontend."""\n\n    goal: str = Field(..., min_length=3, max_length=160)\n    audience: PlanAudience\n    created_at: datetime = Field(default_factory=datetime.utcnow)\n    version: str = Field(default="1.0.0")\n    steps: list[PlanStep] = Field(default_factory=list, min_length=1)\n    risks: list[str] = Field(default_factory=list)\n\n    @model_validator(mode="after")\n    def _ensure_unique_titles(self) -> "Plan":\n        """Guarantee that step titles stay unique for accessible rendering."""\n\n        seen: set[str] = set()\n        for step in self.steps:\n            if step.title in seen:\n                raise ValueError("Step titles must be unique within a plan.")\n            seen.add(step.title)\n        return self\n\n\nclass PlanRequest(BaseModel):\n    """Payload accepted from the UI when requesting a plan."""\n\n    goal: str = Field(..., min_length=3, max_length=160)\n    audience_role: str = Field(..., min_length=2, max_length=64)\n    audience_experience: Literal["beginner", "intermediate", "advanced"]\n    primary_risk: str | None = Field(default=None, max_length=160)\n\n\nclass PlanValidationResult(BaseModel):\n    """Response returned after validating or repairing arbitrary payloads."""\n\n    plan: Plan\n    repaired: bool = Field(default=False)\n    messages: list[str] = Field(default_factory=list)\n')
print('Planner schemas refreshed.')

### Step 2: Planner service
Implement `app/services/planner.py` to generate audience-aware plans and repair malformed payloads.

In [ ]:
from pathlib import Path

service_path = Path('ai-web/backend/app/services/planner.py')
service_path.parent.mkdir(parents=True, exist_ok=True)
service_path.write_text('"""Planner service helpers that keep structured JSON responses consistent."""\n\nfrom __future__ import annotations\n\nfrom typing import Any\n\nfrom pydantic import ValidationError\n\nfrom app.schemas.planner import Plan, PlanRequest, PlanValidationResult, PlanStep\n\n_DEFAULT_STEP_LIBRARY = [\n    {\n        "title": "Clarify problem statement",\n        "description": "Facilitate a working session to confirm the problem this plan solves and document constraints.",\n        "owner": "Product Manager",\n        "duration_minutes": 45,\n        "acceptance_criteria": [\n            "Problem statement documented",\n            "Constraints captured in shared doc",\n        ],\n    },\n    {\n        "title": "Review technical architecture",\n        "description": "Pair with the tech lead to outline service boundaries, APIs, and deployment considerations.",\n        "owner": "Tech Lead",\n        "duration_minutes": 60,\n        "acceptance_criteria": [\n            "Architecture diagram published",\n            "Dependencies reviewed",\n        ],\n    },\n    {\n        "title": "Create validation checklist",\n        "description": "Draft QA scenarios and monitoring hooks to validate the release before and after launch.",\n        "owner": "QA Lead",\n        "duration_minutes": 50,\n        "acceptance_criteria": [\n            "Checklist shared with QA",\n            "Observability tasks assigned",\n        ],\n    },\n]\n\n_BEGINNER_STEP = {\n    "title": "Schedule enablement workshop",\n    "description": "Host a walkthrough to prepare early adopters and gather final questions before launch.",\n    "owner": "Developer Advocate",\n    "duration_minutes": 40,\n    "acceptance_criteria": [\n        "Workshop invite sent",\n        "Feedback doc created",\n    ],\n}\n\n_ADVANCED_STEP = {\n    "title": "Execute load and chaos rehearsal",\n    "description": "Run stress, failover, and rollback drills so production is protected for senior users.",\n    "owner": "Site Reliability",\n    "duration_minutes": 75,\n    "acceptance_criteria": [\n        "Load test report archived",\n        "Rollback path verified",\n    ],\n}\n\n\ndef build_plan(request: PlanRequest) -> Plan:\n    """Create a structured plan tailored for the requested audience."""\n\n    goal = request.goal.strip()\n    audience_role = request.audience_role.strip()\n    steps = _compose_steps(goal, request.audience_experience)\n\n    risks: list[str] = []\n    if request.primary_risk:\n        cleaned_risk = request.primary_risk.strip()\n        if cleaned_risk:\n            risks.append(cleaned_risk)\n\n    plan = Plan(\n        goal=goal,\n        audience={"role": audience_role, "experience_level": request.audience_experience},\n        steps=[PlanStep(**step) for step in steps],\n        risks=risks,\n    )\n    return plan\n\n\ndef validate_plan_payload(payload: dict[str, Any]) -> PlanValidationResult:\n    """Validate or repair arbitrary plan JSON coming from external systems."""\n\n    try:\n        plan = Plan.model_validate(payload)\n        return PlanValidationResult(\n            plan=plan,\n            repaired=False,\n            messages=["Plan payload validated without changes."],\n        )\n    except ValidationError as exc:\n        repaired_payload = _repair_payload(payload)\n        try:\n            repaired_plan = Plan.model_validate(repaired_payload)\n        except ValidationError as follow_up:\n            raise ValueError("Unable to repair plan payload. Please review the submitted structure.") from follow_up\n\n        error_messages = [\n            "Original payload failed validation and was automatically repaired.",\n            *[\n                f"{\'.\'.join(str(part) for part in error[\'loc\'])}: {error[\'msg\']}"\n                for error in exc.errors()\n            ],\n        ]\n        return PlanValidationResult(plan=repaired_plan, repaired=True, messages=error_messages)\n\n\ndef _compose_steps(goal: str, experience_level: str) -> list[dict[str, Any]]:\n    """Generate plan steps based on the target audience."""\n\n    steps: list[dict[str, Any]] = []\n    for template in _DEFAULT_STEP_LIBRARY:\n        updated = dict(template)\n        updated["description"] = template["description"].replace("this plan", f\'"{goal}"\')\n        steps.append(updated)\n\n    if experience_level == "beginner":\n        steps.insert(1, dict(_BEGINNER_STEP))\n    elif experience_level == "advanced":\n        steps.append(dict(_ADVANCED_STEP))\n\n    for idx, step in enumerate(steps, start=1):\n        step.setdefault("acceptance_criteria", [])\n        step.setdefault("owner", "Project Lead")\n        step.setdefault("duration_minutes", 45)\n        step["title"] = step["title"].strip()\n        step["description"] = step["description"].strip()\n        step.setdefault(\n            "acceptance_criteria",\n            [f"Step {idx} outputs documented"],\n        )\n    return steps\n\n\ndef _repair_payload(payload: dict[str, Any]) -> dict[str, Any]:\n    """Attempt to coerce a loosely structured payload into a valid plan."""\n\n    cleaned_goal = str(payload.get("goal", "Unspecified goal")).strip() or "Unspecified goal"\n\n    audience_data = payload.get("audience")\n    if isinstance(audience_data, dict):\n        role = str(audience_data.get("role", "Cross-functional team")).strip() or "Cross-functional team"\n        experience = audience_data.get("experience_level", "intermediate")\n    else:\n        role = "Cross-functional team"\n        experience = "intermediate"\n\n    if experience not in {"beginner", "intermediate", "advanced"}:\n        experience = "intermediate"\n\n    raw_steps = payload.get("steps")\n    cleaned_steps: list[dict[str, Any]] = []\n    if isinstance(raw_steps, list):\n        for index, item in enumerate(raw_steps, start=1):\n            if not isinstance(item, dict):\n                continue\n            title = str(item.get("title") or f"Step {index}").strip() or f"Step {index}"\n            description = str(item.get("description") or f"Document progress for {title}.").strip()\n            owner = str(item.get("owner") or "Project Lead").strip() or "Project Lead"\n            duration = item.get("duration_minutes")\n            try:\n                duration_int = int(duration)\n            except (TypeError, ValueError):\n                duration_int = 45\n            duration_int = max(5, min(duration_int, 240))\n\n            criteria = item.get("acceptance_criteria")\n            if not isinstance(criteria, list):\n                criteria = []\n            cleaned_criteria = [str(entry).strip() for entry in criteria if str(entry).strip()]\n            if not cleaned_criteria:\n                cleaned_criteria = [f"Document completion of {title}."]\n\n            cleaned_steps.append(\n                {\n                    "title": title,\n                    "description": description or f"Document progress for {title}.",\n                    "owner": owner,\n                    "duration_minutes": duration_int,\n                    "acceptance_criteria": cleaned_criteria,\n                }\n            )\n\n    if not cleaned_steps:\n        cleaned_steps = _compose_steps(cleaned_goal, "intermediate")\n\n    risks = payload.get("risks")\n    cleaned_risks: list[str] = []\n    if isinstance(risks, list):\n        cleaned_risks = [str(item).strip() for item in risks if str(item).strip()][:3]\n\n    repaired_payload = {\n        "goal": cleaned_goal,\n        "audience": {"role": role, "experience_level": experience},\n        "steps": cleaned_steps,\n        "risks": cleaned_risks,\n    }\n    return repaired_payload\n')
print('Planner service module updated.')

### Step 3: Planner router
Expose planner functionality through FastAPI routes under the `/planner` prefix.

In [ ]:
from pathlib import Path

router_path = Path('ai-web/backend/app/routers/planner.py')
router_path.parent.mkdir(parents=True, exist_ok=True)
router_path.write_text('"""Planner endpoints powering structured JSON exercises in Lab 04."""\n\nfrom __future__ import annotations\n\nfrom typing import Any\n\nfrom fastapi import APIRouter, HTTPException\n\nfrom app.schemas.planner import Plan, PlanRequest, PlanValidationResult\nfrom app.services.planner import build_plan, validate_plan_payload\n\nrouter = APIRouter(prefix="/planner", tags=["planner"])\n\n\n@router.post("/plan", response_model=Plan)\ndef generate_plan(payload: PlanRequest) -> Plan:\n    """Return a structured plan tailored to the requested audience."""\n\n    return build_plan(payload)\n\n\n@router.post("/plan/validate", response_model=PlanValidationResult)\ndef validate_plan(payload: dict[str, Any]) -> PlanValidationResult:\n    """Validate or repair arbitrary plan JSON provided by the caller."""\n\n    try:\n        return validate_plan_payload(payload)\n    except ValueError as exc:  # Surface repair failures as HTTP 422 errors.\n        raise HTTPException(status_code=422, detail=str(exc)) from exc\n')
print('Planner router available.')

### Step 4: Register the router with FastAPI
Update `app/main.py` so the new planner endpoints are included alongside existing routers.

In [ ]:
from pathlib import Path

main_path = Path('ai-web/backend/app/main.py')
text = main_path.read_text()
if 'planner_router' not in text:
    text = text.replace("from app.routers.echo import router as echo_router\nfrom app.routers.gemini import router as gemini_router\n",
        "from app.routers.agent import router as agent_router\nfrom app.routers.echo import router as echo_router\nfrom app.routers.gemini import router as gemini_router\nfrom app.routers.planner import router as planner_router\n")
    text = text.replace("app.include_router(echo_router)\napp.include_router(gemini_router)\n",
        "app.include_router(agent_router)\napp.include_router(echo_router)\napp.include_router(gemini_router)\napp.include_router(planner_router)\n")
    main_path.write_text(text)
    print('Planner router registered in FastAPI app.')
else:
    print('Planner router already configured.')

### Step 5: Sample payload repair
Try the validation helper with intentionally messy JSON to observe the repair flow.

In [ ]:
from app.services.planner import validate_plan_payload

invalid_payload = {
    'goal': '  Improve onboarding  ',
    'audience': {'role': '', 'experience_level': 'expert'},
    'steps': [
        {'title': ' ', 'description': '', 'acceptance_criteria': 'done'},
        'not even a dict',
    ],
    'risks': [''],
}

result = validate_plan_payload(invalid_payload)
result.plan.model_dump()

## Validation / acceptance checks
```bash
# locally
curl -X POST http://localhost:8000/planner/plan \
  -H 'Content-Type: application/json' \
  -d '{"goal":"Launch curriculum pathways","audience_role":"Instructor","audience_experience":"intermediate"}'

curl -X POST http://localhost:8000/planner/plan/validate \
  -H 'Content-Type: application/json' \
  -d '{"goal":"bad","steps":[{"title":"","description":""}]}'
```
- The first request returns a nested plan with `audience`, `steps`, and optional `risks` fields.
- The second request reports that the payload was repaired and returns a sanitized plan JSON document.
- React development mode renders the planner data without console warnings.

## Homework / extensions
- Extend `PlanStep` with additional metadata (e.g., `dependencies`, `status`) and update the repair logic accordingly.
- Persist validated plans to a database table or document store for auditing.
- Wire the new endpoints into the frontend so students can preview structured plans directly in the app.